In [32]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
print(tf.__version__)
import warnings
warnings.filterwarnings('ignore')

2.11.0


In [33]:
from tensorflow import keras
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences

In [34]:
df = pd.read_csv('train_data_fixed.csv')
numConversion = ["left", "right", "center", "auth", "liberal"]
#lambda function to convert x to lowercase and strip all spaces
df['mainlean'] = df.mainlean.apply(lambda x: numConversion.index(x.lower().strip()))
df.dtypes

tweet         object
mainlean       int64
secondlean    object
dtype: object

In [35]:
df.head(1)

,tweet,mainlean,secondlean
0,Jokes about Alec Baldwin arent funny imo Yeah ...,3,Right


In [36]:
split = round(len(df)*0.8)
train_tweets = df['tweet'][:split]
train_label = df['mainlean'][:split]
test_tweets = df['tweet'][split:]
test_label = df['mainlean'][split:]

In [37]:
training_sentences = []
training_labels = []
testing_sentences = []
testing_labels = []
for row in train_tweets:
    training_sentences.append(str(row))
for row in train_label:
    training_labels.append((row))
for row in test_tweets:
    testing_sentences.append(str(row))
for row in test_label:
    testing_labels.append((row))

In [38]:
print((training_labels[0]))

3


In [39]:
vocab_size = 60000
embedding_dim = 18
max_length = 280
trunc_type = 'post'
oov_tok = '<OOV>'
padding_type = 'post'

In [40]:
tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index

In [41]:
sequences = tokenizer.texts_to_sequences(training_sentences)
padded = pad_sequences(sequences, maxlen=max_length, truncating=trunc_type)
testing_sentences = tokenizer.texts_to_sequences(testing_sentences)
testing_padded = pad_sequences(testing_sentences, maxlen=max_length)

In [42]:
import math
print(math.sqrt(max_length*5))

37.416573867739416


In [43]:
tf.keras.backend.clear_session()
es_callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1)
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=int(max_length)),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(120, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(5, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 280, 18)           1080000   
                                                                 
 global_average_pooling1d (G  (None, 18)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dropout (Dropout)           (None, 18)                0         
                                                                 
 dense (Dense)               (None, 120)               2280      
                                                                 
 dropout_1 (Dropout)         (None, 120)               0         
                                                                 
 dense_1 (Dense)             (None, 5)                 605       
                                                        

In [44]:
training_labels_final = np.array(training_labels)
testing_labels_final = np.array(testing_labels)
poliIdent = model.fit(padded, training_labels_final, epochs=8, validation_data=(testing_padded, testing_labels_final), callbacks=[es_callback])

Epoch 1/8
3525/3525 [==============================] - 31s 9ms/step - loss: 0.6220 - accuracy: 0.6692 - val_loss: 0.4641 - val_accuracy: 0.7697
Epoch 2/8
3525/3525 [==============================] - 28s 8ms/step - loss: 0.4001 - accuracy: 0.8321 - val_loss: 0.4113 - val_accuracy: 0.8074
Epoch 3/8
3525/3525 [==============================] - 29s 8ms/step - loss: 0.3341 - accuracy: 0.8654 - val_loss: 0.4269 - val_accuracy: 0.8043


In [45]:
model.save("poliIdent")

INFO:tensorflow:Assets written to: poliIdent\assets


INFO:tensorflow:Assets written to: poliIdent\assets
